# Prediction Difference Computation Between Gender Group

Logic: 10 Folder Prediction on self-model vs average other groups

In [1]:
%pwd

'/mnt/d/OneDrive - Kyushu University/ESG09_Article/Code'

In [2]:
%cd ..

/mnt/d/OneDrive - Kyushu University/ESG09_Article


/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [3]:
from joblib import dump, load
import numpy as np
import os 
import pandas as pd
import random
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

## Load dataset and divide them into subgroup

In [4]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [5]:
Df = pd.read_parquet(Df_Filename)

In [6]:
Df.shape

(1911212, 64)

In [7]:
Df.columns

Index(['wave', 'INCOME_2', 'Cantril_ladder', 'Health_disable', 'Relative_have',
       'Living_standard_change', 'Enough_food', 'Enough_shelter',
       'Well_rested', 'Respected', 'Smile', 'Interesting_thing', 'Enjoyment',
       'Physical_pain', 'Worry', 'Sadness', 'Stress', 'Anger',
       'City_satisficied', 'Economic_change', 'Goodtime_job', 'Sat_pubtran',
       'Sat_road', 'Sat_edu', 'Sat_qualityair', 'Sat_qualitywater',
       'Sat_healthcare', 'Sat_affhouse', 'Sat_oppofriend', 'Good_minorities',
       'Good_gayles', 'Good_immigrants', 'Donated', 'Volunteer',
       'Help_stranger', 'Voice_official', 'Local_police', 'Safety_walk',
       'Stolen', 'Assualted', 'Religion_importance', 'Children_respected',
       'Children_learn', 'Women_respected', 'Sat_dealpoor', 'Sat_perserveenv',
       'Freedom_chooselife', 'Conf_military', 'Conf_judicial',
       'Conf_government', 'Conf_financial', 'Conf_honestyelections',
       'Freedom_media', 'Corruption_business', 'Corruption_governm

In [8]:
Df.sort_index().head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
367305,4,1.351047e+01,6.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,21.0,1.0,1.0,0.0,2.0,1.0,USA
367306,4,2.296780e+06,6.0,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,34.0,1.0,1.0,0.0,2.0,5.0,USA
367307,4,3.512722e+05,8.0,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,44.0,1.0,1.0,0.0,2.0,5.0,USA
367308,4,5.404188e+04,8.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,67.0,1.0,1.0,0.0,1.0,5.0,USA
367309,4,0.000000e+00,8.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,49.0,2.0,1.0,0.0,1.0,1.0,USA


In [9]:
Df = Df.sample(frac=1, random_state=42)

In [10]:
Df.head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,7.0,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
1562723,11,20170.734058,5.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
708695,6,8664.458086,4.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,8.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND


In [11]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

In [12]:
Df.isna().any().any()

False

In [13]:
Df_male = Df.loc[Df['Gender_female']==0, :].drop(columns=['Gender_female'])

In [14]:
Df_female = Df.loc[Df['Gender_female']==1, :].drop(columns=['Gender_female'])

In [15]:
ym = Df_male['Cantril_ladder']
Xm = Df_male.drop(columns=['Cantril_ladder'])

In [16]:
yf = Df_female['Cantril_ladder']
Xf = Df_female.drop(columns=['Cantril_ladder'])

## Prediction of Each Group Based on Different Models

### Male Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [17]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [18]:
bayes_search = load('Results/BayesSearchMale20iter.joblib')

In [19]:
print(f"Best parameters: {bayes_search.best_params_}")
print(f"Best score: {bayes_search.best_score_}")

Best parameters: OrderedDict([('gamma', 0.02879108443127285), ('learning_rate', 0.004087828726824409), ('max_delta_step', 0.00201590245229211), ('max_depth', 14), ('min_child_weight', 0.0040876867696012075), ('n_estimators', 2294), ('reg_alpha', 0.001), ('reg_lambda', 0.4882641456622785), ('subsample', 0.6548716861574313)])
Best score: 0.38270165821017105


In [20]:
dict(bayes_search.best_params_)

{'gamma': 0.02879108443127285,
 'learning_rate': 0.004087828726824409,
 'max_delta_step': 0.00201590245229211,
 'max_depth': 14,
 'min_child_weight': 0.0040876867696012075,
 'n_estimators': 2294,
 'reg_alpha': 0.001,
 'reg_lambda': 0.4882641456622785,
 'subsample': 0.6548716861574313}

In [21]:
male_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, kwargs = dict(bayes_search.best_params_))

In [22]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [25]:
scores = cross_val_score(male_xgb, Xm, ym, cv=kf, scoring='r2')

/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:43:45] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [26]:
scores

array([0.36891713, 0.37316563, 0.36791924, 0.37675385, 0.37126017,
       0.36903237, 0.37158388, 0.36793965, 0.36704113, 0.37343446])

In [27]:
Xm.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
708695,6,8664.458086,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND
2375401,16,6999.233312,2.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,19.0,1.0,6.0,0.0,2.0,4.0,THA


In [28]:
ym.head()

1469152    7.0
708695     4.0
1828455    2.0
968859     8.0
2375401    5.0
Name: Cantril_ladder, dtype: float64

In [29]:
random.seed(42)

In [30]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [31]:
X = Xm
y = ym
model = male_xgb

In [32]:
f_prediction_list = [np.array(yf.index), np.array(yf.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2f_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2f_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Female Predict
        yf_pred = model.predict(Xf)
        f_prediction_list.append(yf_pred)
        r2f = r2_score(yf, yf_pred)
        r2f_scores.append(r2f)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Female list: {r2f_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2f_scores_list.append(r2f_scores)

Mean R2: 0.3703882746974569
Standard Deviation of R2: 0.001855994816793711
R2 list: [0.3712905727902699, 0.36968465764771696, 0.37078336412363755, 0.3692198350739814, 0.3692851173303684, 0.37146144674349313, 0.3736493156472871, 0.37279533482338845, 0.36787025010185226, 0.3678428526925742]
R2 Female list: [0.3540907201685294, 0.3539787286876783, 0.3541354849950258, 0.35358701169461515, 0.35343316573832184, 0.35414561562634705, 0.35373925087730795, 0.3544838515911062, 0.35328117740771026, 0.35388037648057236]
Mean R2: 0.3704666922360576
Standard Deviation of R2: 0.0016834074894110783
R2 list: [0.3690195981285844, 0.3697122504986856, 0.36937847171576943, 0.3713981472368635, 0.3709158023975425, 0.37277046509477973, 0.36706969886550245, 0.37139256735955717, 0.37017048842424005, 0.37283943263905117]
R2 Female list: [0.35390623947971, 0.3538229295183206, 0.3533476605537983, 0.3532208645196355, 0.3538285992514365, 0.3541331788914994, 0.3538294507788109, 0.3538554346838141, 0.353810402351565, 0

### Male Model Female Prediction

Prediction is average of 10 fold * 10 times

In [42]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
female_preidiction_df = pd.DataFrame(np.array(f_prediction_list).T, columns = column_list)

In [43]:
female_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1562723.0,5.0,5.611097,5.962116,5.251924,5.594006,5.341272,5.342565,5.543282,5.294114,...,5.665200,5.895948,5.396536,5.512073,6.122376,5.671428,5.529995,5.647024,5.543520,5.627588
1,723699.0,6.0,6.141004,6.333287,6.343757,6.281771,6.372932,6.060524,6.280087,6.362103,...,6.200157,6.105019,6.255121,6.486056,6.209045,6.135273,6.258525,5.903100,6.219853,6.275604
2,1535061.0,2.0,4.240648,3.865000,3.974711,3.655260,3.726858,4.103837,3.760774,3.971534,...,4.100354,3.629898,4.065355,4.033564,4.472950,4.042153,4.071698,3.770390,3.890726,4.269206
3,1938251.0,2.0,3.968759,4.231794,3.988369,4.112975,3.992446,3.895156,3.863471,3.839089,...,4.122129,3.663398,3.874006,3.676681,3.847443,3.953981,3.816538,4.098837,3.764192,3.779602
4,1836183.0,10.0,7.291238,7.162137,7.171918,6.889061,7.267672,7.105515,7.201451,6.989357,...,7.259892,7.225546,7.141300,7.175527,6.928216,7.031858,6.889035,7.182191,7.130524,7.223544


In [44]:
female_preidiction_df['Predict_y'] = female_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [45]:
female_preidiction_df['Predict_y'].head()

0    5.587430
1    6.218081
2    4.002232
3    3.880771
4    7.110442
Name: Predict_y, dtype: float64

In [46]:
female_preidiction_df = female_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [47]:
male_preidiction_df['index'] = male_preidiction_df['index'].astype(int)

In [48]:
male_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1562723,5.0,5.587430
1,723699,6.0,6.218081
2,1535061,2.0,4.002232
3,1938251,2.0,3.880771
4,1836183,10.0,7.110442
5,1653464,3.0,6.242113
6,776224,7.0,5.741743
7,718598,5.0,5.396567
8,1701982,7.0,6.311467
9,1578430,7.0,6.311763


In [49]:
female_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMaleModelFemalePrediction_v1.parquet'))

### Male Model Male Prediction

In [51]:
merged_male_prediction_df = None
for male_prediction_df in fold_predictions_array_list:
    male_prediction_df['index'] = male_prediction_df['index'].astype(int)
    male_prediction_df['Real_y'] = male_prediction_df['Real_y'].astype(int)
    print(male_prediction_df.columns)
    if merged_male_prediction_df is None:
        merged_male_prediction_df = male_prediction_df
    else:
        merged_male_prediction_df = pd.merge(merged_male_prediction_df, male_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [52]:
merged_male_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1322632,7,4.472158,4.522396,4.679275,4.549003,4.618958,4.632960,4.301579,4.692605,4.055626,4.697102
1,2218620,5,6.608396,6.605908,6.177011,6.451942,6.325380,6.571249,6.221906,6.455285,6.156618,6.554644
2,563995,2,3.736247,3.617651,3.594791,3.863182,3.645808,3.214748,3.189225,3.617996,3.471398,3.417854
3,1467908,2,3.831100,4.055220,3.472385,3.403348,3.525921,3.483180,3.600840,3.465894,3.770206,3.663028
4,1205253,5,6.429534,6.300669,6.598856,6.566459,6.622838,6.272727,6.466076,6.399439,6.665994,6.724700


In [53]:
merged_male_prediction_df['Predict_y'] = merged_male_prediction_df.iloc[:,2:13].mean(axis = 1)

In [54]:
merged_male_prediction_df = merged_male_prediction_df[['index', 'Real_y', 'Predict_y']]

In [55]:
merged_male_prediction_df.head(10)

,index,Real_y,Predict_y
0,1322632,7,4.522166
1,2218620,5,6.412834
2,563995,2,3.536890
3,1467908,2,3.627112
4,1205253,5,6.504729
5,1084244,8,5.922596
6,2252652,3,5.135747
7,2530259,7,7.109478
8,1995045,2,5.030109
9,2554842,4,6.795876


In [56]:
merged_male_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMaleModelMalePrediction_v1.parquet'))

### Female Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [57]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [58]:
bayes_search = load('Results/BayesSearchFemale20iter.joblib')

In [59]:
print(f"Best parameters: {bayes_search.best_params_}")
print(f"Best score: {bayes_search.best_score_}")

Best parameters: OrderedDict([('gamma', 0.02879108443127285), ('learning_rate', 0.004087828726824409), ('max_delta_step', 0.00201590245229211), ('max_depth', 14), ('min_child_weight', 0.0040876867696012075), ('n_estimators', 2294), ('reg_alpha', 0.001), ('reg_lambda', 0.4882641456622785), ('subsample', 0.6548716861574313)])
Best score: 0.38270165821017105


In [60]:
dict(bayes_search.best_params_)

{'gamma': 0.02879108443127285,
 'learning_rate': 0.004087828726824409,
 'max_delta_step': 0.00201590245229211,
 'max_depth': 14,
 'min_child_weight': 0.0040876867696012075,
 'n_estimators': 2294,
 'reg_alpha': 0.001,
 'reg_lambda': 0.4882641456622785,
 'subsample': 0.6548716861574313}

In [61]:
female_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, kwargs = dict(bayes_search.best_params_))

In [62]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [63]:
scores = cross_val_score(female_xgb, Xf, yf, cv=kf, scoring='r2')

In [64]:
scores

array([0.36799356, 0.37149945, 0.36679217, 0.36619458, 0.36723788,
       0.36890181, 0.37073796, 0.36995464, 0.3693691 , 0.36934261])

In [65]:
Xf.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1562723,11,20170.734058,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
723699,6,15287.346147,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,28.0,2.0,6.0,1.0,2.0,2.0,RUS
1535061,10,4467.534252,1.0,1.0,-0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,69.0,5.0,6.0,0.0,3.0,3.0,MNE
1938251,13,7202.172618,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,47.0,3.0,1.0,0.0,3.0,2.0,BIH
1836183,12,52643.260274,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,62.0,1.0,6.0,0.0,2.0,5.0,MLT


In [66]:
yf.head()

1562723     5.0
723699      6.0
1535061     2.0
1938251     2.0
1836183    10.0
Name: Cantril_ladder, dtype: float64

In [67]:
random.seed(42)

In [68]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [69]:
X = Xf
y = yf
model = female_xgb

In [70]:
m_prediction_list = [np.array(ym.index), np.array(ym.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2m_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2m_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Female Predict
        ym_pred = model.predict(Xm)
        m_prediction_list.append(ym_pred)
        r2m = r2_score(ym, ym_pred)
        r2m_scores.append(r2m)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Male list: {r2m_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2m_scores_list.append(r2m_scores)

Mean R2: 0.3687548402685453
Standard Deviation of R2: 0.002476798169386908
R2 list: [0.36846376282178095, 0.3704658899970298, 0.36925427431622515, 0.3637113163912191, 0.3697625401964818, 0.3673616891402016, 0.3725473023265743, 0.3655401420490716, 0.37080176968784107, 0.36963971575902776]
R2 Male list: [0.3557196066334827, 0.3555643451525735, 0.35584418777971616, 0.3557304455603254, 0.35541768318234324, 0.3559737597905456, 0.3556280998634159, 0.3560954580482597, 0.35549485114351653, 0.35611979526017334]
Mean R2: 0.3688862847672447
Standard Deviation of R2: 0.0021478499942719767
R2 list: [0.3680125383390359, 0.36670963937256507, 0.369806023215418, 0.3719777809769578, 0.3699830980562535, 0.3658337624564678, 0.3719493940204597, 0.3661575849810863, 0.36806311030326266, 0.37036991595094026]
R2 Male list: [0.3561106540721315, 0.3555146709038014, 0.35578746225733915, 0.3559796491465709, 0.35582788482598626, 0.35599877751610143, 0.35617379237467695, 0.3558160536821988, 0.3559336379197524, 0.356

### Female Model Male Prediction

Prediction is average of 10 fold * 10 times

In [71]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
male_preidiction_df = pd.DataFrame(np.array(m_prediction_list).T, columns = column_list)

In [72]:
male_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1469152.0,7.0,7.305373,7.077616,7.029443,6.847871,7.102046,7.270255,6.899257,7.254554,...,7.267439,7.433094,7.132969,7.009861,7.219190,7.079041,7.141830,7.146196,6.988912,7.020994
1,708695.0,4.0,4.632365,4.544305,4.606365,4.785388,4.775038,4.617414,4.735450,4.615685,...,4.408753,4.892575,4.418744,4.323250,4.638156,4.809880,4.959417,4.551565,4.664425,4.914359
2,1828455.0,2.0,6.752893,6.629790,6.766423,6.927818,6.754467,6.822162,6.836002,6.856282,...,6.538272,6.672246,7.069242,6.846605,6.527147,6.921275,6.878062,6.573021,6.864224,6.683884
3,968859.0,8.0,5.742814,5.476525,5.539338,5.308682,5.463693,5.794307,5.302822,5.292184,...,5.567352,4.994583,5.595670,5.436077,5.704936,5.497558,5.265756,4.959220,5.400936,5.810305
4,2375401.0,5.0,5.828291,6.256931,5.844155,5.540818,6.197835,6.016509,5.747581,5.715486,...,6.286999,5.931212,6.241789,6.206881,5.875700,5.900109,5.909745,5.718591,5.616968,5.906798


In [73]:
male_preidiction_df['Predict_y'] = male_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [74]:
male_preidiction_df['Predict_y'].head()

0    7.080436
1    4.719041
2    6.791786
3    5.371026
4    5.912444
Name: Predict_y, dtype: float64

In [75]:
male_preidiction_df = male_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [76]:
male_preidiction_df['index'] = male_preidiction_df['index'].astype(int)

In [77]:
male_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7.0,7.080436
1,708695,4.0,4.719041
2,1828455,2.0,6.791786
3,968859,8.0,5.371026
4,2375401,5.0,5.912444
5,831522,5.0,3.343935
6,2069375,2.0,3.018706
7,1655669,10.0,3.919101
8,1398218,10.0,7.654831
9,1637923,2.0,3.325010


In [78]:
male_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbFemaleModelMalePrediction_v1.parquet'))

### Female Model Female Prediction

In [79]:
merged_female_prediction_df = None
for female_prediction_df in fold_predictions_array_list:
    female_prediction_df['index'] = female_prediction_df['index'].astype(int)
    female_prediction_df['Real_y'] = female_prediction_df['Real_y'].astype(int)
    print(female_prediction_df.columns)
    if merged_female_prediction_df is None:
        merged_female_prediction_df = female_prediction_df
    else:
        merged_female_prediction_df = pd.merge(merged_female_prediction_df, female_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [80]:
merged_female_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,520447,5,5.424016,4.969878,5.425950,5.398604,5.093354,4.813152,5.185786,5.281285,5.351176,4.831287
1,1398320,8,7.185381,7.367653,7.488006,7.455295,7.260145,7.341007,7.312518,7.318009,7.353762,7.536679
2,1767387,5,4.751626,4.731005,4.414942,4.683492,4.844096,4.085826,4.810469,4.484376,4.383704,4.530856
3,2097259,10,4.607144,4.616738,4.581135,4.595199,5.240974,4.604528,4.193696,4.587017,5.090348,4.895068
4,624137,3,3.534788,3.983338,3.465433,3.800375,3.585778,3.377163,3.473117,3.708426,3.600949,3.821369


In [81]:
merged_female_prediction_df['Predict_y'] = merged_female_prediction_df.iloc[:,2:13].mean(axis = 1)

In [82]:
merged_female_prediction_df = merged_female_prediction_df[['index', 'Real_y', 'Predict_y']]

In [83]:
merged_female_prediction_df.head(10)

,index,Real_y,Predict_y
0,520447,5,5.177449
1,1398320,8,7.361845
2,1767387,5,4.572039
3,2097259,10,4.701185
4,624137,3,3.635074
5,947435,8,7.374383
6,554737,7,7.270303
7,2279649,4,6.771342
8,2151240,6,7.463960
9,1172285,6,6.655036


In [84]:
merged_female_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbFemaleModelFemalePrediction_v1.parquet'))